In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = ""

# State and Output Schema

In [2]:
from typing import TypedDict

from langchain_core.pydantic_v1 import BaseModel, Field


# Pydantic model to structure the LLM's output
class ScorePrediction(BaseModel):
    """The structured output for the resume score."""

    score: float = Field(description="The matching score between JD and Resume (0–10)")
    explanation: str = Field(description="The explanation of the matching score")


# The state that will be passed through the graph
class GraphState(TypedDict):
    """The state of our graph."""

    jd: str
    resume: str
    prediction: ScorePrediction  # The final output will be stored here

/Users/nhannguyen/Documents/HCMUS/LLM/Code/llm-resume-evaluation/.env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


# Nodes

In [3]:
import re

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

# The complete, optimized prompt generated by DSPy
OPTIMIZED_PROMPT_TEMPLATE = """
# SYSTEM INSTRUCTION
Your input fields are:
1. `jd` (str): The job description text
2. `resume` (str): The candidate's resume text
Your output fields are:
1. `score` (str): The matching score between JD and Resume (0–10)
2. `explanation` (str): The explanation of the matching score
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## jd ## ]]
{{jd}}

[[ ## resume ## ]]
{{resume}}

[[ ## score ## ]]
{{score}}

[[ ## explanation ## ]]
{{explanation}}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given a job description and a resume, predict a matching score (0-10).

# FEW-SHOT EXAMPLE 1
[[ ## jd ## ]]
Software Engineer needed with experience in Git, Docker, Java, REST APIs. Hit summer discussion culture measure ever.

[[ ## resume ## ]]
Experienced professional skilled in System Design, past, Java, foot. Thank case rather generation inside. Raise new structure race.

[[ ## score ## ]]
4

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 2
[[ ## jd ## ]]
ML Engineer needed with experience in Python, PyTorch, Computer Vision, TensorFlow, Keras. Could chair beautiful social both few through. Entire card much rate politics their identify. Pass sing goal during be those.

[[ ## resume ## ]]
Experienced professional skilled in Computer Vision, Python, MLOps, PyTorch, TensorFlow, parent. Exactly section network detail. Short out team author deal hospital able.

[[ ## score ## ]]
10

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 3
[[ ## jd ## ]]
ML Engineer needed with experience in TensorFlow, Computer Vision, Keras, PyTorch, MLOps. Approach wish fine near. Agree long behind stuff how positive tree. Quality team general office painting official.

[[ ## resume ## ]]
Experienced professional skilled in Cloud, interest, MLOps, Keras, hotel, PyTorch. Address guy fund window well impact quite. Place raise really feeling vote per. Catch board present market society fight foreign. Out generation beyond six degree stop.

[[ ## score ## ]]
8

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FEW-SHOT EXAMPLE 4
[[ ## jd ## ]]
Data Scientist needed with experience in Deep Learning, Machine Learning, NLP, SQL, Pandas, Python, Statistics. Nearly computer close garden. Law individual business hair show. Final though kitchen purpose five. Use sea right civil.

[[ ## resume ## ]]
Experienced professional skilled in Python, Deep Learning, garden, Statistics, SQL, Pandas, stock. Note operation despite born. Step take share million message long board.

[[ ## score ## ]]
8

[[ ## explanation ## ]]
Not supplied for this particular example.

[[ ## completed ## ]]

# FINAL QUERY
[[ ## jd ## ]]
{jd}

[[ ## resume ## ]]
{resume}

Respond with the corresponding output fields, starting with the field `[[ ## score ## ]]`, then `[[ ## explanation ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.
"""


def scorer_node(state: GraphState):
    """Invokes the LLM to score the resume against the job description."""

    model = ChatGoogleGenerativeAI(model="gemma-3n-e2b-it", temperature=0)

    # Create the prompt from the template
    prompt = ChatPromptTemplate.from_template(OPTIMIZED_PROMPT_TEMPLATE)

    # Create the chain
    chain = prompt | model

    # Get inputs from the state
    jd = state["jd"]
    resume = state["resume"]

    # Invoke the chain
    response = chain.invoke({"jd": jd, "resume": resume})
    response_text = response.content

    # Parse the structured output from the model's response
    try:
        score_match = re.search(r"\[\[ ## score ## \]\]\s*([\d.]+)", response_text)
        explanation_match = re.search(
            r"\[\[ ## explanation ## \]\]\s*([\s\S]*?)\s*\[\[ ## completed ## \]\]",
            response_text,
        )

        if not score_match or not explanation_match:
            raise ValueError("Could not find score or explanation tags in the output.")

        score_val = float(score_match.group(1).strip())
        explanation_val = explanation_match.group(1).strip()

        prediction = ScorePrediction(score=score_val, explanation=explanation_val)

    except (ValueError, AttributeError) as e:
        print(f"Error parsing LLM output: {e}\nResponse: {response_text}")
        # Assign a default low score on parsing failure
        prediction = ScorePrediction(
            score=0.0, explanation="Failed to parse model output."
        )

    return {"prediction": prediction}

/Users/nhannguyen/Documents/HCMUS/LLM/Code/llm-resume-evaluation/.env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Graph

In [4]:
from langgraph.graph import END, StateGraph

# Initialize the graph
workflow = StateGraph(GraphState)

# Add the scorer node
workflow.add_node("scorer", scorer_node)

# Set the entry point and the final edge
workflow.set_entry_point("scorer")
workflow.add_edge("scorer", END)

# Compile the graph into a runnable application
app = workflow.compile()

print("\n✅ LangGraph App Compiled Successfully!")

# --- Example Usage ---
jd_example = "Product Manager needed with experience in User Stories, Product Roadmap, Stakeholder Management, Agile."
resume_example = "Experienced professional skilled in blood, Scrum, skill, Stakeholder Management, quality, Agile."

inputs = {"jd": jd_example, "resume": resume_example}

# Run the graph
result = app.invoke(inputs)

print("\n--- Example Result ---")
print(f"Score: {result['prediction'].score}")
print(f"Explanation: {result['prediction'].explanation}")

E0000 00:00:1760460369.734198 25855223 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



✅ LangGraph App Compiled Successfully!

--- Example Result ---
Score: 10.0
Explanation: The resume explicitly lists "Stakeholder Management" and "Agile" which are key requirements in the job description. The resume also mentions "Scrum" which is a popular Agile framework. The skills listed in the resume are highly relevant to the responsibilities of a Product Manager.


# Metrics

In [5]:
import numpy as np
from sklearn.metrics import mean_absolute_error


def accuracy_at_threshold(gold_scores, pred_scores, threshold):
    gold_scores = np.array(gold_scores)
    pred_scores = np.array(pred_scores)
    absolute_errors = np.abs(gold_scores - pred_scores)
    within_threshold_count = np.sum(absolute_errors <= threshold)
    return within_threshold_count / len(gold_scores)


# Evaluate

In [6]:
import pandas as pd

train_df = pd.read_csv("../../data/train.csv")
val_df = pd.read_csv("../../data/test.csv")

## Kaggle dataset

In [7]:
import time

from tqdm import tqdm

gold_scores = []
pred_scores = []

print(f"\n--- Starting Evaluation on {len(val_df)} examples ---")

for i, example in tqdm(
    val_df.iterrows(), desc="Evaluating LangGraph", total=len(val_df)
):
    # Get ground truth data
    jd = example["job_description"]
    resume = example["resume"]
    gold_score = example["match_score"]

    # Run the LangGraph app
    inputs = {"jd": jd, "resume": resume}
    try:
        result = app.invoke(inputs)
        pred_score = result["prediction"].score
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(10)

print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")


--- Starting Evaluation on 3000 examples ---


Evaluating LangGraph:   2%|▏         | 50/3000 [10:34<10:29:13, 12.80s/it]E0000 00:00:1760461007.653735 25855223 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
9

[[ ## explanation ## ]]
The resume lists TensorFlow and Computer Vision, which are directly mentioned in the job description. It also mentions PyTorch, which is also mentioned in the job description. The resume also lists Keras, which is mentioned in the job description. The resume also lists Python, which is mentioned in the job description. The resume also lists SQL and Pandas, which are not mentioned in the job description. The resume also lists Computer Vision, which is mentioned in the job description. The resume also lists TensorFlow, which is mentioned in the job description. The resume also lists Keras, which is mentioned in the job description. The resume also lists PyTorch, which is mentioned in the job description. The resume also lists employee, which is not mentioned in the job description. The resume also lists stock, which is not mentioned in the job descripti

Evaluating LangGraph:   8%|▊         | 236/3000 [50:15<9:56:46, 12.95s/it] E0000 00:00:1760463388.871818 25855223 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
6

[[ ## explanation ## ]]
The resume lists Power BI, which is a key skill mentioned in the job description. However, the resume also lists "pay" which is not relevant to the job description. The resume also lists "they" which is not a relevant skill. The resume also lists "hotel civil" which is not relevant to the job description. The resume also lists "under building respond material arm" which is not relevant to the job description. The resume also lists "perhaps" which is not relevant to the job description. The resume also lists "window adult perhaps country really" which is not relevant to the job description. The resume also lists "than world set adult because natural your" which is not relevant to the job description. The resume also lists "reduce room carry let figure bill" which is not relevant to the job description. The resume also lists "window adult perhaps countr

...

Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
9

[[ ## explanation ## ]]
The resume lists "Agile" and "Scrum" which directly match the requirements of the Product Manager role. It also mentions "User Stories" which is a key skill for the role. The experience in "Agile" and "Scrum" is a strong indicator of suitability. The resume also mentions "User Stories" which is a key skill for the role. The resume also mentions "Agile" and "Scrum" which directly match the requirements of the Product Manager role. The resume also mentions "User Stories" which is a key skill for the role. The resume also mentions "Agile" and "Scrum" which directly match the requirements of the Product Manager role. The resume also mentions "User Stories" which is a key skill for the role. The resume also mentions "Agile" and "Scrum" which directly match the requirements of the Product Manager role. The resume also mentions "User Stories" which is a key 

Evaluating LangGraph:  89%|████████▉ | 2684/3000 [9:39:04<1:05:39, 12.47s/it]E0000 00:00:1760495118.150876 25855223 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Error parsing LLM output: Could not find score or explanation tags in the output.
Response: [[ ## score ## ]]
9

[[ ## explanation ## ]]
The resume lists SQL, Python, Machine Learning, and Statistics, which are all key requirements in the job description. The resume also mentions Machine Learning and Python, which are directly mentioned in the job description. The resume also mentions SQL, which is a requirement. The resume also mentions Machine Learning and Python, which are directly mentioned in the job description. The resume also mentions SQL, which is a requirement. The resume also mentions Deep Learning, which is not mentioned in the job description, but is a related skill. The resume also mentions media, which is not mentioned in the job description. The resume also mentions Deep Learning, which is not mentioned in the job description. The resume also mentions stock, which is not mentioned in the job description. The resume also mentions partner, which is not mentioned in the jo

Evaluating LangGraph: 100%|██████████| 3000/3000 [10:45:35<00:00, 12.91s/it]

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 1.7320
---
🎯 Accuracy@1 (error <= 1.0): 62.77%
🎯 Accuracy@2 (error <= 2.0): 69.83%
🎯 Accuracy@3 (error <= 3.0): 91.70%
--------------------------------------


In [8]:
import json

with open("preds/gemma_kaggle.json", "w") as f:
    json.dump(pred_scores, f)

## Collected dataset

In [9]:
import json
import time

from tqdm import tqdm

collected_folder = "/Users/nhannguyen/ngtuthanhan@gmail.com - Google Drive/My Drive/HCMUS/LLM/Data_Collector_30_8"
matching_json = os.path.join(collected_folder, "Matching_Result.json")

with open(matching_json, "r") as f:
    matching_data = json.load(f)

pred_scores = []
gold_scores = []

for example in tqdm(
    matching_data, desc="Evaluating LangGraph", total=len(matching_data)
):
    jd_path = example["JD"]
    resume_path = example["CV"]
    gold_score = example["Score"]
    with open(os.path.join(collected_folder, "JD", jd_path) + ".txt", "r") as f:
        jd = f.read()
    with open(os.path.join(collected_folder, "CV", resume_path) + ".txt", "r") as f:
        resume = f.read()
    inputs = {"jd": jd, "resume": resume}
    try:
        result = app.invoke(inputs)
        pred_score = result["prediction"].score
    except Exception as e:
        print(f"An error occurred during graph execution: {e}")
        pred_score = 0.0  # Assign a penalty score

    gold_scores.append(gold_score)
    pred_scores.append(pred_score)

    # To avoid rate limiting
    time.sleep(30)


print("--- Evaluation Complete ---")

# --- Calculate Metrics ---

# 1. Mean Absolute Error (MAE)
final_mae = mean_absolute_error(gold_scores, pred_scores)

# 2. Accuracy@threshold
acc_at_1 = accuracy_at_threshold(gold_scores, pred_scores, threshold=1)
acc_at_2 = accuracy_at_threshold(gold_scores, pred_scores, threshold=2)
acc_at_3 = accuracy_at_threshold(gold_scores, pred_scores, threshold=3)


# --- Print Results ---

print("\n--- LangGraph Evaluation Results ---")
print(f"📊 Mean Absolute Error (MAE): {final_mae:.4f}")
print("---")
print(f"🎯 Accuracy@1 (error <= 1.0): {acc_at_1:.2%}")
print(f"🎯 Accuracy@2 (error <= 2.0): {acc_at_2:.2%}")
print(f"🎯 Accuracy@3 (error <= 3.0): {acc_at_3:.2%}")
print("--------------------------------------")

Evaluating LangGraph: 100%|██████████| 87/87 [51:30<00:00, 35.52s/it]

--- Evaluation Complete ---

--- LangGraph Evaluation Results ---
📊 Mean Absolute Error (MAE): 2.5977
---
🎯 Accuracy@1 (error <= 1.0): 34.48%
🎯 Accuracy@2 (error <= 2.0): 52.87%
🎯 Accuracy@3 (error <= 3.0): 73.56%
--------------------------------------


In [10]:
import json

with open("preds/gemma_collected.json", "w") as f:
    json.dump(pred_scores, f)